In [5]:
# !pip install awswrangler

In [7]:
import boto3
import awswrangler as wr
import pandas as pd
# from lightgbm import LGBMClassifier

boto3.setup_default_session(region_name="eu-north-1")

In [8]:
# df_campaign_history = wr.athena.read_sql_query('SELECT * FROM "customerone_mock_data_rl"."campaign_history" limit 10;', 
#                               database="customerone_mock_data_rl",
#                               workgroup="dev-ds-athena-workgroup")
# df_campaign_history

In [9]:
df_master = wr.athena.read_sql_query('SELECT * FROM "customerone_mock_data_rl"."master";', 
                              database="customerone_mock_data_rl",
                              workgroup="dev-ds-athena-workgroup")
df_master

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0
0,100110,2021-08-09,<NA>,51,77,F,Mercadobury,Gabon,4,Bad,...,1.000000,NaN,NaN,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed
1,100121,2021-06-28,<NA>,32,68,U,Lake Anthonymouth,Madagascar,1,Good,...,NaN,NaN,1.0,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
2,10013,2021-10-11,<NA>,58,73,M,Jodiport,Haiti,3,Bad,...,NaN,NaN,0.0,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
3,10014,2021-06-30,<NA>,20,30,F,Lake Daniel,Ethiopia,1,Bad,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
4,100171,2021-08-17,<NA>,68,56,M,Griffithland,Serbia,2,Bad,...,1.666667,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40204,100928,2021-08-22,<NA>,44,36,M,West Shelleyland,United Arab Emirates,1,Bad,...,1.000000,1.387097,1.0,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40205,100955,2021-08-05,<NA>,53,11,U,East Jamesstad,Guinea-Bissau,1,Good,...,2.818182,NaN,1.0,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40206,100957,2021-05-03,<NA>,33,46,M,Lucaston,Vanuatu,2,Good,...,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,xsell_cust_voice_to_fixed
40207,100982,2021-07-12,<NA>,64,20,U,West Michael,Yemen,0,Bad,...,NaN,NaN,1.0,NaN,0,0,0.000000,0.0,0.0,xsell_cust_voice_to_fixed


In [11]:
# 74 columns
[print(f"\n** {col}:\n{df[col].value_counts(dropna=False)} \n") for col in df.columns];

Index(['customer_id', 'current_dt', 'tgt_xsell_cust_voice_to_fixed',
       'dmgrphc_b_age_bucket_txt', 'dmgrphc_b_zip_code_cd',
       'dmgrphc_b_gender_typ', 'dmgrphc_b_city_txt', 'dmgrphc_b_country_txt',
       'dmgrphc_b_no_of_dependents_val', 'dmgrphc_b_habits_desc_txt',
       'dmgrphc_b_lifestyle_desc_txt', 'dmgrphc_d_tenure_in_days_val',
       'ci_d_sum_total_compl_last_0_to_30_days_cnt',
       'ci_d_sum_total_compl_last_0_to_60_days_cnt',
       'ci_d_sum_total_num_of_interactions_last_0_to_30_days_cnt',
       'ci_d_sum_total_num_of_interactions_last_0_to_60_days_cnt',
       'ci_d_sum_total_resolved_cmpl_last_0_to_30_days_val',
       'ci_d_sum_total_resolved_cmpl_last_0_to_60_days_val',
       'ci_d_compl_last_0_to_30_avg_days_settle_cnt',
       'ci_d_sum_compl_fraga_last_0_to_30_days_cnt',
       'ci_d_sum_compl_fraga_last_0_to_60_days_cnt',
       'ci_d_sum_compl_incident_last_0_to_30_days_cnt',
       'ci_d_sum_compl_incident_last_0_to_60_days_cnt',
       'ci_d_total

In [ ]:
# df_master['tgt_xsell_cust_voice_to_fixed'].plot()
# plt.show();

# 

# questions!

what is lead_time_window & target_window in yml file? 

/Users/sarah/VS_projects/cvm-customerone/conf/base/parameters/telco/next_best_action/xsell/xsell_cust_fixed_to_voice_conf.yml

```python
{{model}}.master.target:
    target_variable_column: tgt_{{model}}
    lead_time_window: 1d
    target_window: 45d
```


# 1- preprocessing

## 1-1 exclude_features?

In [ ]:
# training config
{{model}}.training.exclude_features:
  exact_match:
    - "subscription_status_ind"

## 1-2 sampling? 

In [ ]:
{{model}}.training.mst_sampling:
  #positive_sample: 1.0
  #negative_sample: 1.0
  max_sample_size: 1000000
  random_state: 42

## 1-3 splitting_params?

In [ ]:
{{model}}.training.splitting_params:
  splitter: customerone.pipelines.telco.test_train_splitter.OutOfTimeSplitter
  splitter_args: # Arguments to be passed into the splitter, see the splitter docs for details
    n_splits: 1 # The n_splits argument controls the number of iterations
    test_size: 0.2
    date_col: "current_dt"
    verbose: 1
    random_state: 42
  group_col_name: "customer_id" # column be useded as a group vector, if the selected spliter uses the group parameter. See the splitter docs for details
  iteration_col_name: iteration_id # Name of the iteration ID column to be added
  split_col_name: split # Name of the TRAIN/TEST/etc. split column

## 1-4 subsplitting_params

In [ ]:
{{model}}.training.subsplitting_params:
  splitter: sklearn.model_selection.StratifiedShuffleSplit
  splitter_args: # Arguments to be passed into the splitter, see the splitter docs for details
    test_size: 0.1
    random_state: 42 # n_splits is fixed to be 1 for sub-splitters. Any higher value will be ignored
  source_split: TRAIN # The set to split
  target_splits: # The two sets to split the source set into. Must be two targets
    - TRAIN
    - CAL

## 1.5 imputing_params

```python
{{model}}.training.imputation_params:
  num_imputer: # Name of the first transformer
    transformer: sklearn.impute.SimpleImputer
    transformer_args: # Arguments to be passed into the transformer, see the transformer docs for details
      verbose: 1
      strategy: mean
  cat_imputer: # Name of the second imputer. We can specify as many as we like, but one feature must only be used by one imputer
    transformer: sklearn.impute.SimpleImputer
    transformer_args:
      verbose: 1
      strategy: most_frequent
      missing_values: null
```

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
class PandasSimpleImputer(SimpleImputer):
    """
    A wrapper around `SimpleImputer` to return data frames with columns.
    """

    def fit(self, X, y=None):
        self.columns = X.columns
        return super().fit(X, y)

    def transform(self, X):
        return pd.DataFrame(super().transform(X), columns=self.columns, index=X.index)


# define imputer
imputer = SimpleImputer(strategy='mean')
# fit on the dataset
imputer.fit(X)
# transform the dataset
Xtrans = imputer.transform(X)
# print total missing
print('Missing: %d' % sum(isnan(Xtrans).flatten()))


num_imputer = SimpleImputer(strategy='mean', verbose=1)
cat_imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan, verbose=1)



## 1-6 encoding_params

In [ ]:
{{model}}.training.encoding_params:
  ordinal_encoder:
    transformer: category_encoders.target_encoder.TargetEncoder
    target: "tgt_{{model}}"
    transformer_args:
      cols: # columns to encode, defaults to all

In [ ]:
{{model}}.training.preprocessing:
  categorical:
    impute:
      class: customerone_lib.common.pkgs.predictive_modeling.preprocessing.PandasSimpleImputer
      args:
        strategy: most_frequent
    encoder:
      class: category_encoders.target_encoder.TargetEncoder
  numerical:
    imputer:
      class: customerone_lib.common.pkgs.predictive_modeling.preprocessing.PandasSimpleImputer
      args:
        strategy: mean

# 2- modeling

In [ ]:
{{model}}.training.estimator:
  pipeline: imblearn.pipeline.Pipeline
  args:
    steps:
      over_sampler: 
        class: imblearn.over_sampling.SMOTE
        args:
          random_state: 42
      estimator: 
        class: lightgbm.LGBMClassifier

In [ ]:
# training pipeline

pipelines = [
        cdl_to_edge,
        determine_data_types,
        data_splitter,
        preprocessing,
        estimator,
        tune_hyperparameters,
        fit_models,
        calibrate_model,
        predictions,
        performance,
        mlflow_save_model,
    ]

### 2-1-1 cdl_to_edge sample_training_df

### 2-1-2 cdl_to_edge drop_invalid_features

In [ ]:
# /Users/sarah/VS_projects/cvm-customerone/src/customerone_lib/common/pkgs/feature_selection/drop_features.py

def drop_invalid_features(data) -> pd.DataFrame:
   """
   Drop constant, null and nonexisting columns from dataframe

   Args:
      data: input dataframe, could be pandas or spark

   Returns:
      dataframe with the invalid columns dropped
   """
   features = data.columns
   if isinstance(data, pd.DataFrame):
      valid_features = [c for c in features if len(data[c].dropna().unique()) > 1]
      df = data[valid_features]
   elif isinstance(data, pyspark.sql.DataFrame):
      distinct_count = data.agg(*(f.approx_count_distinct(f.col(c)).alias(c) for c in features)).toPandas()
      valid_features = distinct_count.columns[distinct_count.loc[0,:] > 1]
      df = data.select(*valid_features)
   else:
      logger.error(f"Invalid data type, expected pd.DataFrame or spark.DataFrame but got: {type(data)}")

   invalid_features = set(features) - set(valid_features)
   if invalid_features:
      logger.warning(f"Dropping {len(invalid_features)} invalid features: {invalid_features}")

   return df

### 2-1-3 cdl_to_edge filter_features

In [ ]:
def filter_features(data, params: Dict[str, Any]) -> pd.DataFrame:
   """
   Drop constant, null and nonexisting columns from dataframe

   Args:
      data: input dataframe, could be pandas or spark

   Returns:
      dataframe with the invalid columns dropped
   """
   exact_match = params.get("exact_match")
   regex_match = params.get("regex")
   features = data.columns

   invalid_features = []
   if exact_match:
      invalid_features += [c for c in features if c in to_list(exact_match)]
   if regex_match:
      for regex in to_list(regex_match):
         r = re.compile(regex)
         invalid_features += list(filter(r.match, features))
   
   # remove duplicates
   invalid_features = set(invalid_features)
   valid_features = list(set(features) - invalid_features)

   if invalid_features:
      logger.warning(f"Filtering out {len(invalid_features)} features: {invalid_features}")

   return data[valid_features] if isinstance(data, pd.DataFrame) else data.select(*valid_features)

### 2-1-4 node spark_utils.to_pandas

In [ ]:
def to_pandas(
    data: DataFrame,
    pandas_params: Dict[str, Any],
    spine_params: Dict[str, Any],
    target_params: Dict[str, Any],
    data_dictionary: Optional[DataDict] = None,
    features: List[str] = None,
    ref_date: Optional[str] = None,
) -> pd.DataFrame:
    """
    Applies toPandas to a spark DataFrame and optionally filters:
        - columns by keeping only relevant features for a model
        - rows by selecting only a specific reference date
    Args:
        data (spark.sql.DataFrame): the input spark DataFrame
        pandas_params:
            ref_period (Optional[str]): the reference period used to filter
                resulting dataframe - specified in the format
                [0-9]+(d|w|m). example: 1d for 1 day, 3m for 3 months, 12m for 12
                months and 5w for 5 weeks.
                Used in 2 ways:
                    * if include_target = True, set max date to last day when target
                      is available and filter period from
                      `max(date when target not nan) - ref_period` to `max(date when
                      target not nan)`
                    * if include_target = False, filter following period: from
                      `max(date) - ref_period` to `max(date)`
                Not possible to specify together with ref_date.
            include_target (Optional[bool]): when true, the resulting dataframe
                will include observations for which the value of the target variable
                is either True or False (i.e., observations in the perimeter of the
                model). Moreover, if the DataDict exposes the column
                `use_case:target_switch`, then those observations that switch during
                the hold out period (meaning their value of target will be 0)
                will be filtered out as well.
        data_dictionary (DataDict): the use case data dictionary used to export
            only relevant columns (i.e., features)
        features (List[str]): export only relevant feature columns (i.e., features)
        ref_date (Union[str, pd.Timestamp]): the reference date used to filter
            the resulting DataFrame. Not possible to specify together with
            ref_period.
    Returns:
         a pandas DataFrame
    Raises:
        ValueError: This is raised when the specified `ref_date` in the params
            cannot be cast into pd.Datetime
        AttributeError: This is raised when both ref_date and ref_period are
            specified at the same time, only one of these arguments should be
            specified at a time.
    """
    ref_period = pandas_params.get("ref_period")
    include_target = pandas_params.get("include_target")
    key_col = to_list(spine_params.get('keys'))
    date_col = spine_params.get('date_column')
    target_col = target_params.get('target_variable_column')

    if ref_date and ref_period:
        raise AttributeError(
            "Cannot specify both ref_date and ref_period at"
            "the same time. Specify only one argument."
        )

    feat_col, target_switch_col = [None] * 2

    if data_dictionary or features:
        if data_dictionary:
            feat_col = data_dictionary.get_features()
            target_switch_col = data_dictionary.get_target_switch_variable()
        elif features:
            feat_col = features

        export_cols = []
        if date_col:
            export_cols += [date_col]
        if key_col:
            export_cols += key_col
        export_cols += feat_col
        if include_target and target_col:
            export_cols += [target_col]

        cols_diff = set(export_cols) - set(data.columns)
        if len(cols_diff) != 0:
            logger.warning(f"The following columns are not available in the DataFrame and will be ignored: {cols_diff}")
            export_cols = list(set(export_cols) - cols_diff)

        data = data.select(export_cols)

    if ref_date and date_col:
        try:
            if not re.match(r"\d{4}-\d{2}-\d{2}", ref_date):
                raise ValueError("Date format should follow yyyy-mm-dd.")

            ref_date = pd.to_datetime(ref_date, infer_datetime_format=True).strftime(
                "%Y-%m-%d"
            )
            data = data.filter(f.col(date_col) == ref_date)
        except (ValueError, TypeError) as exception:
            raise exception("'ref_date' type casting failed") from exception

    if include_target and target_col:
        data = data.filter(f.col(target_col).isNotNull())

    if include_target and target_switch_col:
        data = data.filter(~f.col(target_switch_col))

    if ref_period and date_col:
        data = filter_ref_window(data, ref_period, date_col)

    return data.toPandas()


In [ ]:
len(trained_models) = 1, 
trained_models[0] = 
{'feature_col_names': ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg', 'dmgrphc_b_zip_code_cd', 'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg', 'ci_d_sum_total_compl_last_0_to_60_days_cnt', 'ci_d_sum_total_num_of_interactions_last_0_to_30_days_cnt', 'cust_prd_hld_d_fixedbroadband_active_0_to_30_days_avg_val', 'cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val', 'rev_m_eom_total_bill_amt_m0_val', 'ci_d_sum_total_num_of_interactions_last_0_to_60_days_cnt', 'cust_prd_hld_d_fixedbroadband_active_0_to_30_over_30_to_60_days_avg_val', 'rev_m_eom_total_bill_amt_m1_val', 'ci_d_compl_last_0_to_30_avg_days_settle_cnt', 'ci_d_sum_total_resolved_cmpl_last_0_to_30_days_val', 'cust_prd_hld_d_mobilevoicesubscription_closed_30_to_60_days_avg_val', 'cust_prd_hld_d_postpaid_mobilevoicesubscription_cnt', 'cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val', 'cust_prd_hld_d_postpaid_tvchannelpackage_cnt', 'ci_d_sum_cmpl_over_contacts_in_0_60_days', 'cust_prd_hld_d_vas_active_0_to_30_days_avg_val', 'dmgrphc_b_age_bucket_txt', 'cust_prd_hld_d_postpaid_vas_cnt', 'cust_prd_hld_d_vas_closed_val', 'cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val', 'cust_prd_hld_d_postpaid_fixedbroadband_cnt', 'cust_prd_hld_d_vas_closed_0_to_30_days_avg_val', 'cust_prd_hld_d_tvchannelpackage_active_30_to_60_days_avg_val', 'ci_d_sum_cmpl_over_contacts_in_0_30_days', 'cust_prd_hld_d_vas_active_0_to_30_over_30_to_60_days_avg_val', 'cust_prd_hld_d_fixedbroadband_closed_val', 'ci_d_sum_compl_fraga_last_0_to_30_days_cnt', 'cust_prd_hld_d_fixedbroadband_closed_0_to_30_days_avg_val', 'cust_prd_hld_d_tvchannelpackage_closed_val', 'ci_d_sum_compl_incident_last_0_to_60_days_cnt', 'cust_prd_hld_d_vas_active_30_to_60_days_avg_val', 'cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val', 'cust_prd_hld_d_vas_active_val', 'cust_prd_hld_d_mobilevoicesubscription_closed_val', 'ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt', 'cust_prd_hld_d_mobilevoicesubscription_active_0_to_30_days_avg_val', 'ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt', 'cust_prd_hld_d_tvchannelpackage_closed_30_to_60_days_avg_val', 'cust_prd_hld_d_mobilevoicesubscription_active_0_to_30_over_30_to_60_days_avg_val', 'cust_prd_hld_d_mobilevoicesubscription_active_30_to_60_days_avg_val', 'cust_prd_hld_d_fixedbroadband_closed_0_to_30_over_30_to_60_days_avg_val', 'dmgrphc_b_no_of_dependents_val', 'ci_d_total_num_of_interactions_weekday_last_0_to_30_cnt', 'ci_d_sum_compl_incident_last_0_to_30_days_cnt', 'ci_d_sum_cmpl_resolved_over_contacts_in_0_60_days', 'dmgrphc_d_tenure_in_days_val', 'ci_d_variation_sum_total_resolved_cmpl_last_30_to_60_days_val', 'cust_prd_hld_d_tvchannelpackage_active_0_to_30_days_avg_val', 'cust_prd_hld_d_tvchannelpackage_active_val', 'cust_prd_hld_d_tvchannelpackage_closed_0_to_30_over_30_to_60_days_avg_val', 'cust_prd_hld_d_fixedbroadband_active_val', 'cust_prd_hld_d_fixedbroadband_closed_30_to_60_days_avg_val', 'ci_d_variation_sum_total_compl_last_30_over_60_days_cnt', 'ci_d_sum_total_resolved_cmpl_last_0_to_60_days_val', 'ci_d_sum_total_compl_last_0_to_30_days_cnt', 'ci_d_sum_cmpl_resolved_over_contacts_in_0_30_days', 'cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_days_avg_val', 'cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val', 'ci_d_sum_compl_fraga_last_0_to_60_days_cnt', 'cust_prd_hld_d_vas_closed_30_to_60_days_avg_val', 'cust_prd_hld_d_mobilevoicesubscription_active_val', 'rev_m_eom_total_bill_amt_m2_val'], 'estimator_name': None, 'estimator_args': {'preprocessing__categorical__encoder__smoothing': 5.0, 'preprocessing__categorical__encoder__min_samples_leaf': 1000, 'over_sampler__sampling_strategy': 0.25, 'estimator__subsample_freq': 1, 'estimator__subsample': 1.0, 'estimator__reg_lambda': 0.1, 'estimator__reg_alpha': 0.1, 'estimator__num_leaves': 50, 'estimator__n_estimators': 150, 'estimator__min_split_gain': 0.0, 'estimator__min_child_weight': 0.01, 'estimator__min_child_samples': 100, 'estimator__max_depth': 15, 'estimator__learning_rate': 0.15, 'estimator__colsample_bytree': 1.0, 'estimator__class_weight': [], 'estimator__boosting_type': 'gbdt'}, 'splits_to_fit': 'TRAIN', 'weight_samples': False, 'model': Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   PandasSimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder(min_samples_leaf=1000,
                                                                                 smoothing=5.0))]),
                                                  ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                                   'dmgrphc_b_zip_code_cd',
                                                   'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_...
                 CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=LGBMClassifier(class_weight=[],
                                                                                                                                                                                                                                                                                                                                                learning_rate=0.15,
                                                                                                                                                                                                                                                                                                                                                max_depth=15,
                                                                                                                                                                                                                                                                                                                                                min_child_samples=100,
                                                                                                                                                                                                                                                                                                                                                min_child_weight=0.01,
                                                                                                                                                                                                                                                                                                                                                n_estimators=150,
                                                                                                                                                                                                                                                                                                                                                num_leaves=50,
                                                                                                                                                                                                                                                                                                                                                reg_alpha=0.1,
                                                                                                                                                                                                                                                                                                                                                reg_lambda=0.1,
                                                                                                                                                                                                                                                                                                                                                subsample_freq=1),
                                                                                                                                                                                                                                                                                                                  cv='prefit'),
                                                                                                                                                                                                                                                                            cv='prefit'),
                                                                                                                                                                                                                                      cv='prefit'),
                                                                                                                                                                                                cv='prefit'),
                                                                                                                                                          cv='prefit'),
                                                                                                                    cv='prefit'),
                                                                              cv='prefit'),
                                        cv='prefit'))]), 'target_col_name': 'tgt_xsell_cust_voice_to_fixed', 'weights': None, 'calibration_method': None, 'feature_importances_': array([   0, 3550,    0,    0,    0,    0], dtype=int32), 'fitted_model': Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   PandasSimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   TargetEncoder(min_samples_leaf=1000,
                                                                                 smoothing=5.0))]),
                                                  ['rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg',
                                                   'dmgrphc_b_zip_code_cd',
                                                   'rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_...
                 CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=LGBMClassifier(class_weight=[],
                                                                                                                                                                                                                                                                                                                                                learning_rate=0.15,
                                                                                                                                                                                                                                                                                                                                                max_depth=15,
                                                                                                                                                                                                                                                                                                                                                min_child_samples=100,
                                                                                                                                                                                                                                                                                                                                                min_child_weight=0.01,
                                                                                                                                                                                                                                                                                                                                                n_estimators=150,
                                                                                                                                                                                                                                                                                                                                                num_leaves=50,
                                                                                                                                                                                                                                                                                                                                                reg_alpha=0.1,
                                                                                                                                                                                                                                                                                                                                                reg_lambda=0.1,
                                                                                                                                                                                                                                                                                                                                                subsample_freq=1),
                                                                                                                                                                                                                                                                                                                  cv='prefit'),
                                                                                                                                                                                                                                                                            cv='prefit'),
                                                                                                                                                                                                                                      cv='prefit'),
                                                                                                                                                                                                cv='prefit'),
                                                                                                                                                          cv='prefit'),
                                                                                                                    cv='prefit'),
                                                                              cv='prefit'),
                                        cv='prefit'))]), 'calibrated_model': CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=CalibratedClassifierCV(base_estimator=LGBMClassifier(class_weight=[],
                                                                                                                                                                                                                                                                                         learning_rate=0.15,
                                                                                                                                                                                                                                                                                         max_depth=15,
                                                                                                                                                                                                                                                                                         min_child_samples=100,
                                                                                                                                                                                                                                                                                         min_child_weight=0.01,
                                                                                                                                                                                                                                                                                         n_estimators=150,
                                                                                                                                                                                                                                                                                         num_leaves=50,
                                                                                                                                                                                                                                                                                         reg_alpha=0.1,
                                                                                                                                                                                                                                                                                         reg_lambda=0.1,
                                                                                                                                                                                                                                                                                         subsample_freq=1),
                                                                                                                                                                                                                                                           cv='prefit'),
                                                                                                                                                                                                                     cv='prefit'),
                                                                                                                                                                               cv='prefit'),
                                                                                                                                         cv='prefit'),
                                                                                                   cv='prefit'),
                                                             cv='prefit'),
                       cv='prefit')}